In [ ]:
# auto reload

%load_ext autoreload
%autoreload 2

In [ ]:
from dsd import DATA_DIR
from dsd.diffusion_rendering import *
from dsd.cropped_diffusion_rendering import *

In [ ]:
image_dir = DATA_DIR / "renders/mugs/100/Threshold_porcelain_Coffee_Mug_All_over_bead/018"
input_images = DiffusionRenderInputImages.from_render_dir(image_dir)
#input_images.depth_image[input_images.depth_image > 1.0] = 1.0


In [ ]:
import matplotlib.pyplot as plt
def visualize_results(images):
    # plot original image and depth, as well as the processed images
    fig, axes = plt.subplots(4,4, figsize=(20, 20))
    axes = axes.flatten()
    for i, image in enumerate(images):
        ax = axes[i]
        ax.imshow(np.array(image))
    plt.show()

In [ ]:
strength = 1.0
controlnet_conditioning_scale = 1.3
n_steps = 50

In [ ]:
renderer = SD15RealisticCheckpointControlNetFromDepthRenderer(num_inference_steps=n_steps,controlnet_conditioning_scale=controlnet_conditioning_scale,strength=strength)


In [ ]:
cropped_renderer = CroppedRenderer(renderer)

In [ ]:
#inpainter = SD2NormalCheckpointInpaintRenderer(num_images_per_prompt=4, num_inference_steps=n_steps, strength = 1.0)
inpainter = SD2InpaintingRenderer(num_images_per_prompt=4, num_inference_steps=n_steps, strength = 1.0)


In [ ]:
crop_and_inpaint_renderer = CropAndInpaintRenderer(cropped_renderer, inpainter)

In [ ]:
prompt = "a white, ceramic mug with a picture of a dog, natural light, RAW"
background_prompt = "a table in a messy robot lab, natural light, RAW"

In [ ]:

s2_images = crop_and_inpaint_renderer(prompt, background_prompt, input_images)


In [ ]:
stage_1_results = cropped_renderer(prompt, input_images)
stage_2_inputs = DiffusionRenderInputImages(np.copy(input_images.rgb_image), np.copy(input_images.depth_image), np.copy(input_images.mask))
# dilate the mask to inpaint the background
original_mask = np.copy(stage_2_inputs.mask)
stage_2_inputs.mask = cv2.dilate(stage_2_inputs.mask, np.ones((5,5), np.uint8), iterations=1)
# invert the mask to inpaint the background
stage_2_inputs.mask = 1 - stage_2_inputs.mask
# duplicate the input image to create one for each stage 1 result
stage_2_inputs = [copy.deepcopy(stage_2_inputs) for _ in stage_1_results]
# set the input images to the stage 1 results
stage_2_results = []
for i, result in enumerate(stage_1_results):
    stage_2_inputs[i].rgb_image = np.array(result).astype(np.float32)/255.0
    stage_2_result = inpainter(background_prompt, stage_2_inputs[i])
    for j, image in enumerate(stage_2_result):
        image = np.array(image).astype(np.float32)/255.0
        image[original_mask > 0.5] = stage_2_inputs[i].rgb_image[original_mask >  0.5]
        stage_2_result[j] = Image.fromarray((image*255).astype(np.uint8))
    stage_2_results.extend(stage_2_result)


In [ ]:
visualize_results(stage_2_results)